# 텍스트 전처리

**데이터 불러오기(txt를 데이터프레임으로 변환)**

In [1]:
import pandas as pd

In [33]:
curse_data = pd.read_table("/content/drive/Othercomputers/내 컴퓨터/Curse-words_Detection/Curse-detection-data/dataset.txt", names=["text", "label"], sep="|", header=None)
curse_data

,text,label
0,좌배 까는건 ㅇㅂ,1
1,집에 롱 패딩만 세 개다. 10년 더 입어야지 ㅋㅋ,0
2,개소리야 니가 빨갱이를 옹호하고 드루킹을 ㅇㅇ짓이라고 말못해서 삐진거야 빨갱아,1
3,세탁이라고 봐도 된다,0
4,애새끼가 초딩도 아니고 ㅋㅋㅋㅋ,1
...,...,...
5819,좌우 헬파이어 3개씩 6개 장착에 아파치보다 약하지만 20mm 기관포 장착임,0
5820,"세금 내놓으라고 데모질 중 ㅋㅋ간첩, 도둑놈 새끼들이 대통령 해처먹으니까 나도 같...",1
5821,너가 한 말 중에,0
5822,제갈대중 ㅇㅂ,0


**train, validation, test 데이터 분할**

In [3]:
# train:validation:test = 6:2:2 (추후 성능 보고 비율 조정예정)
train_dataset = curse_data.sample(frac=0.6, random_state=2022)  

par_dataset = curse_data.drop(train_dataset.index)  # val, test를 나눌 파라미터
val_dataset = par_dataset.sample(frac=0.5, random_state=2022)   # 0.4 * 0.5 = 0.2로 전체 데이터의 20%
test_dataset = par_dataset.drop(val_dataset.index)

In [4]:
train_dataset

,text,label
3536,서성한 인문상경(O),0
1600,나경원이 얼굴 보지 말자 이제!!,0
4181,이새끼 나이도 존나 마니 처먹은 들딱이더만,1
2131,개선유무 따지고있는데 서울대는 왜나옴?,0
4393,너똥구멍에너으면타이트해?..,1
...,...,...
3068,무죄 무죄 무죄 ~~1!@@~!!,0
553,나뻤당 ㅋㅋㅋ,0
5747,난 김성령 ~,0
5154,이미 알고 있어서 ㅁㅈㅎ주려고 했는데1렙이라 ㅁㅈㅎ 못줘서 속으로 애미 욕함,1


In [5]:
val_dataset

,text,label
304,복싱에선 여태 일본이 많이 쳐발렸다..,1
4823,찌익,0
2132,ㄹㅇ 거기에 아낌없는 지원을한 따블당의원님ㅠㅠ 든든합니다!!,0
3421,게이야..집까지 한시간 가량 소리지르면서 우는거는 민폐아니야..우는 모습도 안귀여울...,1
4094,홍위병들 당할수가 없어,0
...,...,...
2697,지방도 번화가는 잘되더라. 번화가 아닌곳만 뒤지는거지 ㅋㅋ,0
2296,ㅅㅂ유투버새기들 꼴보기싫다유투브 언제망하냐,1
2968,이국종도 지난번에 욕처먹을짓 했잖아이재명 탄원서 제출한거 ㅋㅋㅋ난 그거 보고 아 저...,1
5404,니말대로 2060년에 휴거가 있을수도 있고,0


In [6]:
test_dataset

,text,label
3,세탁이라고 봐도 된다,0
4,애새끼가 초딩도 아니고 ㅋㅋㅋㅋ,1
5,731부대의 후예라 그런지 가학적인 아이디어는 세계최고임 이래서 애교만 떨어도 돈 ...,1
24,난 사이트 마다 아이디/비번 다 다르고 난이도 높음엑셀 파일에 모든 아이디/비번 /...,0
30,군대에서 여군이 필요한가,0
...,...,...
5759,공개해야됨,0
5771,진짜 순발력 센스는 올타임원탑,0
5791,나씨 전라도성씨임,0
5806,이래서 애새끼 함부로 싸지르는거 아니다,1


**전처리 함수**

In [ ]:
!pip install soynlp
!pip install emoji==1.7.0

In [8]:
import re
import emoji
from soynlp.normalizer import repeat_normalize

In [9]:
emojis = ''.join(emoji.UNICODE_EMOJI.keys())
pattern = re.compile(f'[^ .,?!/@$%~％·∼()\x00-\x7Fㄱ-ㅣ가-힣{emojis}]+')
url_pattern = re.compile(
    r'https?:\/\/(www\.)?[-a-zA-Z0-9@:%._\+~#=]{1,256}\.[a-zA-Z0-9()]{1,6}\b([-a-zA-Z0-9()@:%_\+.~#?&//=]*)')

In [10]:
def clean(x):
    x = pattern.sub(' ', x)         # 일반적으로 사용하는 특수문자, 영어, 한글, emoji제외 공백으로 치환
    x = url_pattern.sub('', x)      # URL 제거
    x = x.strip()                   # 문자의 시작과 끝에서 공백제거
    x = repeat_normalize(x, num_repeats=2)      # 반목되는 문자의 축약 횟수 2개로 줄임
    return x